First task

Fully connected neural networks

Data import

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('newDataBaseVk.csv')
data = data.fillna(data.mean())
data = data.iloc[0:None, 1:10]
data.head()

,countFriends,countFollowers,boolComments,countOwnerPosts,countOwnerReposts,countPhotos,countVideos,countLikesPhotoes,sex
0,75,137,False,11,9,4,71,14,2
1,173,117,False,6,14,9,26,234,1
2,324,549,False,20,0,40,64,1561,1
3,678,97,False,1,19,24,240,1111,2
4,398,38,False,14,6,11,12,480,1


Preprocessing

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

allX = data.iloc[0:None, 0:8]
allY = data.iloc[0:None, 8]

allX.boolComments = allX.boolComments.astype('int64')

for i in range(data.shape[0]):
    allY[i] =  allY[i] - 1
    
Xtrain, Xtest, Ytrain, Ytest = train_test_split(allX, allY, test_size=0.2)

scaler = preprocessing.MinMaxScaler()
Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain.values))
Xtest = pd.DataFrame(scaler.transform(Xtest.values))

Keras model

In [3]:
from keras.models import Sequential
from keras.layers import Dense

# Define Keras model
model = Sequential()
model.add(Dense(32, input_dim=8, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit Keras model
model.fit(Xtrain, Ytrain, epochs=100, verbose=0)

# Evaluate Keras model
_, accuracy = model.evaluate(Xtest, Ytest)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/100
25/25 [==============================] - 0s 623us/step - loss: 0.6872 - accuracy: 0.5523
Epoch 2/100
25/25 [==============================] - 0s 623us/step - loss: 0.6841 - accuracy: 0.5706
Epoch 3/100
25/25 [==============================] - 0s 582us/step - loss: 0.6791 - accuracy: 0.5833
Epoch 4/100
25/25 [==============================] - 0s 582us/step - loss: 0.6712 - accuracy: 0.5815
Epoch 5/100
25/25 [==============================] - 0s 620us/step - loss: 0.6625 - accuracy: 0.5863
Epoch 6/100
25/25 [==============================] - 0s 582us/step - loss: 0.6707 - accuracy: 0.5743
Epoch 7/100
25/25 [==============================] - 0s 540us/step - loss: 0.6576 - accuracy: 0.6102
Epoch 8/100
25/25 [==============================] - 0s 582us/step - loss: 0.6538 - accuracy: 0.6061
Epoch 9/100
25/25 [==============================] - 0s 572us/step - loss: 0.6467 - accuracy: 0.6254
Epoch 10/100
25/25 [==============================] - 0s 574us/step - loss: 0.6470 - accura

25/25 [==============================] - 0s 551us/step - loss: 0.5380 - accuracy: 0.7285
Epoch 81/100
25/25 [==============================] - 0s 540us/step - loss: 0.5408 - accuracy: 0.7272
Epoch 82/100
25/25 [==============================] - 0s 540us/step - loss: 0.5346 - accuracy: 0.7449
Epoch 83/100
25/25 [==============================] - 0s 582us/step - loss: 0.5459 - accuracy: 0.7364
Epoch 84/100
25/25 [==============================] - 0s 582us/step - loss: 0.5469 - accuracy: 0.7206
Epoch 85/100
25/25 [==============================] - 0s 582us/step - loss: 0.5505 - accuracy: 0.7301
Epoch 86/100
25/25 [==============================] - 0s 543us/step - loss: 0.5353 - accuracy: 0.7345
Epoch 87/100
25/25 [==============================] - 0s 581us/step - loss: 0.5222 - accuracy: 0.7425
Epoch 88/100
25/25 [==============================] - 0s 539us/step - loss: 0.5414 - accuracy: 0.7179
Epoch 89/100
25/25 [==============================] - 0s 583us/step - loss: 0.5556 - accuracy: 

Second task

Convolutional neural networks

Data import and Preprocessing

In [4]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split

def create_dataset(img_folder):
    img_data_array = []
    class_name = []
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (100, 100), interpolation = cv2.INTER_AREA)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255.0
            img_data_array.append(image)
            class_name.append(dir1)
            
    return img_data_array, class_name

# Extract the image array and class name
img_data, class_name = create_dataset('mask_data/')
target_dict = {k: v for v, k in enumerate(np.unique(class_name))}
target_val = [target_dict[class_name[i]] for i in range(len(class_name))]

img_data = np.array(img_data)
target_val = np.array(target_val)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(img_data, target_val, test_size=0.2)

Keras model

In [5]:
import tensorflow as tf

# Define Keras model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 5, activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile Keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit Keras model
model.fit(Xtrain, Ytrain, epochs=13, verbose=1)

# Evaluate Keras model
_, accuracy = model.evaluate(Xtest, Ytest)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/13
189/189 [==============================] - 17s 90ms/step - loss: 0.4027 - accuracy: 0.8124
Epoch 2/13
189/189 [==============================] - 17s 90ms/step - loss: 0.2560 - accuracy: 0.9008
Epoch 3/13
189/189 [==============================] - 17s 91ms/step - loss: 0.1793 - accuracy: 0.9308
Epoch 4/13
189/189 [==============================] - 17s 91ms/step - loss: 0.1619 - accuracy: 0.9370
Epoch 5/13
189/189 [==============================] - 17s 91ms/step - loss: 0.1138 - accuracy: 0.96002s - - ETA: 0s - loss: 0.1139 - ac
Epoch 6/13
189/189 [==============================] - 18s 93ms/step - loss: 0.1081 - accuracy: 0.9581
Epoch 7/13
189/189 [==============================] - 18s 95ms/step - loss: 0.0611 - accuracy: 0.9782
Epoch 8/13
189/189 [==============================] - 18s 95ms/step - loss: 0.0398 - accuracy: 0.9872
Epoch 9/13
189/189 [==============================] - 18s 94ms/step - loss: 0.0425 - accuracy: 0.9869
Epoch 10/13
189/189 [===========================

Third task

Recurrent neural networks

Data import

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

pd.set_option('max_colwidth', 400)
true = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')

true = true.iloc[0:100, 1:2]
fake = fake.iloc[0:100, 1:2]

true['truth'] = [1 for i in range(true.shape[0])]
fake['truth'] = [0 for i in range(fake.shape[0])]

corpus = pd.concat([true, fake])
allX = corpus['text'].values
allY = corpus['truth'].values

Xtrain, Xtest, Ytrain, Ytest = train_test_split(allX, allY, test_size=0.2)

Preprocessing

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 4000
input_length = 4

tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(Xtrain)

Xtrain = pad_sequences(tokenizer.texts_to_sequences(Xtrain), input_length)
Xtest = pad_sequences(tokenizer.texts_to_sequences(Xtest), input_length)

Keras model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

# Define Keras model
model = Sequential()
model.add(Embedding(num_words, 128, input_length=input_length))
model.add(SimpleRNN(128, return_sequences=True))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Compile Keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit Keras model
model.fit(Xtrain, Ytrain, epochs=13, verbose=1)

# Evaluate Keras model
_, accuracy = model.evaluate(Xtest, Ytest)
print('Accuracy: %.2f' % (accuracy*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            512000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 4, 128)            32896     
_________________________________________________________________
dense_5 (Dense)              (None, 4, 1)              129       
Total params: 545,025
Trainable params: 545,025
Non-trainable params: 0
_________________________________________________________________
Epoch 1/13
5/5 [==============================] - 1s 5ms/step - loss: 0.6842 - accuracy: 0.6088
Epoch 2/13
5/5 [==============================] - 0s 5ms/step - loss: 0.6235 - accuracy: 0.8291
Epoch 3/13
5/5 [==============================] - 0s 5ms/step - loss: 0.5227 - accuracy: 0.9477
Epoch 4/13
5/5 [==============================] - 0s 5ms/step - loss: 0.3600 - accuracy: 0

## 